# Fine-tuning paso a paso: Llama-3.2-1B con AmericasNLP y diccionario estructurado

Este notebook guía el proceso completo y reproducible para ajustar un modelo LLM (meta-llama/Llama-3.2-1B) usando HuggingFace Transformers, con datos de AmericasNLP y un diccionario estructurado.

## 1. Importar librerías necesarias

Instala y carga todas las librerías requeridas para el fine-tuning reproducible.

In [ ]:
%pip uninstall -y torch torchvision torchaudio

In [10]:
import sys
print("Python ejecutable:", sys.executable)

Python ejecutable: c:\Users\walte\miniconda3\envs\python_env\python.exe


In [9]:
%pip uninstall -y transformers accelerate peft sentence-transformers
%pip uninstall -y torch torchvision torchaudio

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
Found existing installation: peft 0.9.0
Uninstalling peft-0.9.0:
  Successfully uninstalled peft-0.9.0
Note: you may need to restart the kernel to use updated packages.


Found existing installation: torch 2.9.1
Uninstalling torch-2.9.1:
  Successfully uninstalled torch-2.9.1
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


In [10]:
%pip install transformers==4.38.2 accelerate==0.27.2 peft==0.9.0 datasets
%pip install torch==2.2.0+cu118 torchvision==0.17.1+cu118 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118

  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
  Using cached peft-0.9.0-py3-none-any.whl.metadata (13 kB)
  Using cached torch-2.9.1-cp312-cp312-win_amd64.whl.metadata (30 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)
Using cached peft-0.9.0-py3-none-any.whl (190 kB)
Using cached torch-2.9.1-cp312-cp312-win_amd64.whl (110.9 MB)

   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]
   ---------------------------------------- 0/4 [torch]


ERROR: Cannot install torch==2.2.0+cu118 and torchvision==0.17.1+cu118 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


Reiniciar Kernels:
Ctrl + Shift + P
Jupyter: Restart Kernel

In [1]:
import transformers, accelerate, peft, torch

print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("PEFT:", peft.__version__)
print("Torch:", torch.__version__)

assert transformers.__version__ == "4.38.2"
assert accelerate.__version__ == "0.27.2"
assert peft.__version__ == "0.9.0"

print("✔ Versiones correctas")


c:\Users\walte\miniconda3\envs\python_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1214 22:25:22.843000 42400 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Transformers: 4.38.2
Accelerate: 0.27.2
PEFT: 0.9.0
Torch: 2.9.1+cpu
✔ Versiones correctas


In [2]:
import random
import numpy as np
import torch

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

print("✔ Imports OK")


✔ Imports OK


In [3]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ CUDA no disponible")

print("✔ Semilla fijada")


⚠️ CUDA no disponible
✔ Semilla fijada


## 2. Cargar y preparar el conjunto de datos

Cargamos los archivos de AmericasNLP y el diccionario estructurado, y preparamos los pares (input, output) para el fine-tuning.

In [5]:
import pandas as pd

# Cargar AmericasNLP (train/dev/test)
train_path = 'AmericasNLP/guarani-train.tsv'
dev_path = 'AmericasNLP/guarani-dev.tsv'
test_path = 'AmericasNLP/guarani-test.tsv'

try:
    df_train = pd.read_csv(train_path, sep='\t')
    df_dev = pd.read_csv(dev_path, sep='\t')
    df_test = pd.read_csv(test_path, sep='\t')
    print('Train:', df_train.shape, '| Dev:', df_dev.shape, '| Test:', df_test.shape)
except Exception as e:
    print('Error cargando AmericasNLP:', e)

# Cargar diccionarios estructurados de la carpeta archivos
dict_ge_path = 'archivos/diccionario_guarani_espanol_estructurado.tsv'
dict_eg_path = 'archivos/diccionario_espanol_guarani_estructurado.tsv'

try:
    df_dict_ge = pd.read_csv(dict_ge_path, sep='\t')
    print('Diccionario guaraní-español:', df_dict_ge.shape)
except Exception as e:
    print('Error cargando diccionario guaraní-español:', e)

try:
    df_dict_eg = pd.read_csv(dict_eg_path, sep='\t')
    print('Diccionario español-guaraní:', df_dict_eg.shape)
except Exception as e:
    print('Error cargando diccionario español-guaraní:', e)

# Ejemplo de los primeros registros de cada diccionario
print('Ejemplo guaraní-español:')
print(df_dict_ge.head())
print('Ejemplo español-guaraní:')
print(df_dict_eg.head())

Train: (178, 4) | Dev: (79, 4) | Test: (364, 4)
Diccionario guaraní-español: (12073, 4)
Diccionario español-guaraní: (15891, 4)
Ejemplo guaraní-español:
  palabra_guarani tipo_etiqueta categoria_gramatical  \
0               a           NaN                  NaN   
1               a           NaN          f. v. ra s.   
2               ã           NaN                 m sa   
3               ã           NaN            adj. dem.   
4               ã           NaN                ta s.   

                                   traduccion_limpia  
0        Vocal que se pronuncia igual que en español  
1      Pref. a. de 1ª. Per. sin. para verbos propios  
2  Vocal que se pronuncia igual que la a pero con...  
3                                              Estos  
4                                              estas  
Ejemplo español-guaraní:
  palabra_espanol tipo_etiqueta categoria_gramatical  \
0               a           NaN          f. va m va.   
1               a           NaN            

In [6]:
# Preprocesar los diccionarios estructurados para tareas de traducción
if 'df_dict_eg' in locals():
    dict_data_eg = df_dict_eg[['palabra_espanol', 'traduccion_limpia']].copy()
    dict_data_eg = dict_data_eg.rename(columns={'palabra_espanol': 'input', 'traduccion_limpia': 'output'})
    print('Ejemplo español-guaraní estructurado:')
    print(dict_data_eg.head())
else:
    print('No se cargó el diccionario español-guaraní.')

if 'df_dict_ge' in locals():
    dict_data_ge = df_dict_ge[['palabra_guarani', 'traduccion_limpia']].copy()
    dict_data_ge = dict_data_ge.rename(columns={'palabra_guarani': 'input', 'traduccion_limpia': 'output'})
    print('Ejemplo guaraní-español estructurado:')
    print(dict_data_ge.head())
else:
    print('No se cargó el diccionario guaraní-español.')

Ejemplo español-guaraní estructurado:
  input                     output
0     a  Pu’ae avañe’ẽmeicha ipúva
1     a                         Pe
2     a                          e
3     a                         ve
4     a                         re
Ejemplo guaraní-español estructurado:
  input                                             output
0     a        Vocal que se pronuncia igual que en español
1     a      Pref. a. de 1ª. Per. sin. para verbos propios
2     ã  Vocal que se pronuncia igual que la a pero con...
3     ã                                              Estos
4     ã                                              estas


In [7]:
# Preprocesar AmericasNLP: crear pares (input, output) para fine-tuning

def build_input(row):
    return f"{row['Source']} [{row['Change']}]"

df_train = df_train.dropna(subset=['Source', 'Change', 'Target'])
df_dev = df_dev.dropna(subset=['Source', 'Change', 'Target'])

train_data = pd.DataFrame({
    'input': df_train.apply(build_input, axis=1),
    'output': df_train['Target']
})
dev_data = pd.DataFrame({
    'input': df_dev.apply(build_input, axis=1),
    'output': df_dev['Target']
})

# Concatenar los datos de los diccionarios estructurados para aumentar el set de entrenamiento
train_data = pd.concat([train_data, dict_data_eg, dict_data_ge], ignore_index=True)

print(train_data.head())

                                               input  \
0  Ha’e ombojerekuri umi kutuhakuéra poro’o [PERS...   
1  Ha’e ombojerekuri umi kutuhakuéra poro’o [TYPE...   
2  Ha’e ombojerekuri umi kutuhakuéra poro’o [ASPE...   
3    Mombe’ukuéra omboty kuri pende arete [TYPE:NEG]   
4  Mombe’ukuéra omboty kuri pende arete [ASPECT:I...   

                                            output  
0       Nde rembojerekuri umi kutuhakuéra tuicháva  
1    Ha’e ndombojereikuri umi kutuhakuéra tuicháva  
2  Ha’e ombojerehina kuri umi kutuhakuéra tuicháva  
3          Mombe’ukuéra ndombotyi kuri pende arete  
4        Mombe’ukuéra omboty kuri hína pende arete  


## 3. Tokenización de los datos

Utilizamos el tokenizer del modelo base para convertir los textos en tensores adecuados para el modelo.

In [8]:
# Cargar el tokenizer de Bloom-560m desde HuggingFace
model_name = 'bigscience/bloom-560m'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenización de los datos de entrenamiento y validación
def tokenize_function(example):
    return tokenizer(example['input'], truncation=True, padding='max_length', max_length=128)

# Convertir a HuggingFace Dataset
from datasets import Dataset, DatasetDict
train_hf = Dataset.from_pandas(train_data)
dev_hf = Dataset.from_pandas(dev_data)

datasets = DatasetDict({
    'train': train_hf,
    'validation': dev_hf
})

tokenized_datasets = datasets.map(tokenize_function, batched=True)
print(tokenized_datasets)

c:\Users\walte\miniconda3\envs\python_env\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 79/79 [00:00<00:00, 8380.97 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask'],
        num_rows: 28142
    })
    validation: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask'],
        num_rows: 79
    })
})


## 4. Configurar el modelo base para fine tuning

Cargamos el modelo preentrenado Llama-3.2-1B y lo preparamos para el ajuste fino.

In [9]:
# Cargar el modelo base para causal language modeling (Bloom-560m)
model = AutoModelForCausalLM.from_pretrained(model_name)

print('Modelo cargado:', model_name)

Modelo cargado: bigscience/bloom-560m


## 5. Definir hiperparámetros y configuración de entrenamiento

Establecemos los parámetros principales para el entrenamiento reproducible.

In [10]:
training_args = TrainingArguments(
    output_dir='./finetuned',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    seed=SEED,
    report_to='none',
    push_to_hub=False
)

print('Hiperparámetros definidos.')

Hiperparámetros definidos.


## 6. Entrenar el modelo

Ejecutamos el proceso de entrenamiento usando los datos tokenizados y la configuración definida.

In [14]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    acc = np.mean([
        p.strip() == l.strip()
        for p, l in zip(decoded_preds, decoded_labels)
    ])

    return {"accuracy": acc}


In [15]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
dev_dataset = Dataset.from_pandas(dev_data)


In [16]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        examples["output"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [17]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

dev_dataset = dev_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dev_dataset.column_names
)


Map: 100%|██████████| 79/79 [00:00<00:00, 6002.83 examples/s]


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
)


In [19]:
trainer.train()


  0%|          | 0/10554 [00:00<?, ?it/s]c:\Users\walte\miniconda3\envs\python_env\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  0%|          | 9/10554 [02:55<53:18:06, 18.20s/it]

KeyboardInterrupt: 

## 7. Evaluar el modelo ajustado

Evaluamos el modelo fine-tuned en el conjunto de validación y mostramos métricas relevantes.

In [ ]:
# Evaluar el modelo en el conjunto de validación
eval_results = trainer.evaluate()
print('Resultados de evaluación:', eval_results)

## 8. Guardar el modelo fine-tuned

Guardamos el modelo entrenado y el tokenizer para su uso posterior o despliegue.

In [ ]:
# Guardar el modelo y el tokenizer
output_dir = './llama3-finetuned-final'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Modelo y tokenizer guardados en {output_dir}')

## 9. Inferencia: probar el modelo fine-tuned

Ejecuta el modelo ajustado sobre ejemplos nuevos o del set de test, como se requiere en la consigna del trabajo final.

In [ ]:
# Ejemplo de inferencia con el modelo fine-tuned
from transformers import pipeline

# Cargar modelo y tokenizer fine-tuned
dir_finetuned = './llama3-finetuned-final'
model = AutoModelForCausalLM.from_pretrained(dir_finetuned)
tokenizer = AutoTokenizer.from_pretrained(dir_finetuned)

# Ejemplo: usar un input del set de test o uno propio
test_example = "Ore ndorombyai kuri [TYPE:AFF]"
inputs = tokenizer(test_example, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=30)
pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
print('Input:', test_example)
print('Predicción:', pred)